In [1]:
import os
import math
import mxnet as mx
from mxnet import image
from mxnet import nd, gluon, autograd, init
from mxnet.gluon import nn
from tensorboardX import SummaryWriter
import numpy as np
import shutil
from mxnet.gluon.data import vision
import pandas as pd

In [2]:
%run resnet.py
%run densenet.py

In [8]:
demo = False
if demo:
    # 注意：此处使用小训练集为便于网页编译。Kaggle的完整数据集应包括5万训练样本。
    train_dir = 'train_tiny'
    # 注意：此处使用小测试集为便于网页编译。Kaggle的完整数据集应包括30万测试样本。
    test_dir = 'test_tiny'
    # 注意：此处相应使用小批量。对Kaggle的完整数据集可设较大的整数，例如128。
    batch_size = 1
else:
    train_dir = 'train'
    test_dir = 'test'
    batch_size = 128

data_dir = '../data/kaggle_cifr-10'
label_file = 'trainLabels.csv'
input_dir = 'train_valid_test'
valid_ratio = 0.1

In [10]:
def transform_train(data, label):
    im = data.asnumpy()
    im = np.pad(im, ((4, 4), (4, 4), (0, 0)), mode='constant', constant_values=0)
    im = nd.array(im, dtype='float32') / 255
    auglist = image.CreateAugmenter(data_shape=(3, 32, 32), resize=0, rand_mirror=True,
                                    rand_crop=True,
                                   mean=np.array([0.4914, 0.4822, 0.4465]),
                                   std=np.array([0.2023, 0.1994, 0.2010]))
    for aug in auglist:
        im = aug(im)
    im = nd.transpose(im, (2, 0, 1)) # channel x width x height
    return im, nd.array([label]).astype('float32')

def transform_test(data, label):
    im = data.astype('float32') / 255
    auglist = image.CreateAugmenter(data_shape=(3, 32, 32), mean=np.array([0.4914, 0.4822, 0.4465]),
                                   std=np.array([0.2023, 0.1994, 0.2010]))
    for aug in auglist:
        im = aug(im)
    im = nd.transpose(im, (2, 0, 1))
    return im, nd.array([label]).astype('float32')

In [11]:
input_str = data_dir + '/' + input_dir + '/'

# 读取原始图像文件。flag=1说明输入图像有三个通道（彩色）。
train_ds = vision.ImageFolderDataset(input_str + 'train', flag=1, 
                                     transform=transform_train)
valid_ds = vision.ImageFolderDataset(input_str + 'valid', flag=1, 
                                     transform=transform_test)
train_valid_ds = vision.ImageFolderDataset(input_str + 'train_valid', 
                                           flag=1, transform=transform_train)
test_ds = vision.ImageFolderDataset(input_str + 'test', flag=1, 
                                     transform=transform_test)

loader = gluon.data.DataLoader

train_data = loader(train_ds, 16, shuffle=True, last_batch='keep')
valid_data = loader(valid_ds, 16, shuffle=True, last_batch='keep')
train_valid_data = loader(train_valid_ds, 128, shuffle=True, last_batch='keep')
test_data = loader(test_ds, 128, shuffle=False, last_batch='keep')

In [12]:
def get_densenet(ctx):
    num_outputs = 10
    net = DenseNet(growthRate=12, depth=100, reduction=0.5, bottleneck=True, nClasses=10)
    net.initialize(ctx=ctx, init=init.Xavier())
    return net

def get_resnet(ctx):
    num_outputs = 10
    net = ResNet164_v2(num_outputs)
    net.initialize(ctx=ctx, init=init.Xavier())
    return net

In [13]:
net1 = get_densenet(mx.gpu(0))
net1.hybridize()
#net1.load_params('./densenet-0.9378.params', ctx=mx.gpu(0))
#net1.load_params('./densenet-254.params', ctx=mx.gpu(0))
#net1.load_params('./densenet-348.params', ctx=mx.gpu(0))
net1.load_params('./densenet-0.9542.params', ctx=mx.gpu(0))

In [14]:
net2 = get_resnet(mx.gpu(0))
net2.hybridize()
net2.load_params('./resnet-0.9544.params', ctx=mx.gpu(0))
#net2.load_params('./resnet164v2-188.params', ctx=mx.gpu(0))
#net2.load_params('./resnet164v2-230.params', ctx=mx.gpu(0))
#net2.load_params('./resnet164v2-274.params', ctx=mx.gpu(0))
#net2.load_params('./resnet164v2-364-0.954091816367-0.999844461726-5e-05-0.0005.params', ctx=mx.gpu(0))
#net2.load_params('./resnet164v2-282.params', ctx=mx.gpu(0))
#net2.load_params('./resnet164v2-324.params', ctx=mx.gpu(0))

## Ensemble networks to predict

In [ ]:
# Combint two networks with average
preds = []
for data, _ in test_data:
    output1 = nd.softmax(net1(data.as_in_context(mx.gpu(0))))
    output2 = nd.softmax(net2(data.as_in_context(mx.gpu(0))))
    output = nd.array((output1 + output2)/2)
    pred_label = output.argmax(axis=1)
    preds.extend(pred_label.astype(int).asnumpy())
print("done prediction")
sorted_ids = list(range(1, len(test_ds) + 1))
sorted_ids.sort(key = lambda x:str(x))

df = pd.DataFrame({'id': sorted_ids, 'label': preds})
df['label'] = df['label'].apply(lambda x: train_ds.synsets[x])
#df.to_csv('submission-densenet-0.9378-resnet-0.9394.csv', index=False)
#df.to_csv('submission-densenet-0.9542-resnet-0.9539-mean.csv', index=False)
df.to_csv('submission-densenet-0.9542-resnet-0.9544-mean.csv', index=False)
print("done submission")

In [ ]:
# Combine two networks with max
preds = []
for data, _ in test_data:
    output1 = nd.softmax(net1(data.as_in_context(mx.gpu(0))))
    output2 = nd.softmax(net2(data.as_in_context(mx.gpu(0))))
    output = nd.concat(*[output1, output2], dim=1)
    pred_label = output.argmax(axis=1) % 10
    preds.extend(pred_label.astype(int).asnumpy())

sorted_ids = list(range(1, len(test_ds) + 1))
sorted_ids.sort(key = lambda x:str(x))

df = pd.DataFrame({'id': sorted_ids, 'label': preds})
df['label'] = df['label'].apply(lambda x: train_ds.synsets[x])
#df.to_csv('submission-densenet-0.9378-resnet-0.9394.csv', index=False)
df.to_csv('submission-densenet-0.9542-resnet-0.9544-max.csv', index=False)
